# 风险溢价

## 加载模块

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import mstats
import statsmodels.api as sm
from numpy.lib.stride_tricks import as_strided as stride

## 读入文件

In [2]:
df_merge = pd.read_csv('../data/stock-accounting.csv')
df_merge['date'] = pd.to_datetime(df_merge['date'])

## Fama-MacBeth 回归

剔除 `nan` 数据

In [10]:
idx_fac = ['beta', 'market-cap', 'book-to-market', 'profit', 'investment', 'past-return']
par = ['const'] + idx_fac
df_fit = df_merge[['PERMNO', 'date', 'stg', 'RET-RF'] + idx_fac].copy()
df_fit['market-cap'] = df_fit['market-cap'].apply(np.log)
df_fit[idx_fac] = df_fit.groupby('PERMNO')[idx_fac].shift(1)
df_fit = df_fit.replace([np.inf, -np.inf], np.nan)
df_fit = df_fit.dropna()
df_fit[idx_fac] = df_fit[idx_fac].apply(mstats.winsorize, limits=[0.01, 0.01])
df_fit.shape[0]

986307

In [13]:
time_series = df_fit['date'].unique()
time_series = np.sort(time_series)

df_params = pd.DataFrame(columns=['date'] + par)

for i, t in enumerate(time_series):
    df_date = df_fit[(df_fit['stg'] != 'Unknown') & (df_fit['date'] == t)].copy()
    x = df_date[idx_fac]
    z = (x - x.mean()) / x.std()
    z = z.clip(-3, 3)
    z = sm.add_constant(z)
    y = df_date['RET-RF']
    result = sm.OLS(y, z).fit()
    df_params.loc[i, 'date'] = t
    df_params.loc[i, par] = result.params
df_params[par].mean() * 100

const             1.032627
beta              0.110572
market-cap       -0.210257
book-to-market    0.446862
profit            0.295573
investment        -0.24141
past-return       0.243864
dtype: object

## 多空投资组合

## 滚动窗口估值

## EXHIBIT 9